In [1]:
import numpy as np
from torch import optim, nn, Tensor
from torch.nn import functional as F
import torch
import wandb
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import transformers
import lightning as L
from inspect import signature, _ParameterKind
import copy
import gc
import datasets
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger

In [2]:
if torch.cuda.get_device_capability()[0] >= 8:
    torch.set_float32_matmul_precision('medium')

In [3]:
wandb.login(key='os.environ[WANDB_API_KEY]', relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/wwu/.netrc


True

In [4]:
class LitUnbalancedModel(L.LightningModule):
    '''
    Train only position encodings.
    '''
    def __init__(
        self,
        model_name='gpt2',
        lr=6e-4,
        num_warmup_steps=1000,
        alpha=0.01,
    ):
        super().__init__()
        args = vars()
        for param in list(signature(LitUnbalancedModel.__init__).parameters)[1:]:
            setattr(self, param, args[param])
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_config(config=config)
        self.save_hyperparameters()

    def forward(self, batch):
        out = self.model.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            use_cache=True,
        )
        ce = nn.CrossEntropyLoss()
        out['even_loss'] = ce(
            out.logits.transpose(1, 2)[...,::2],
            batch['input_ids'][...,1::2]
        )
        out['odd_loss'] = ce(
            out.logits.transpose(1, 2)[...,1:-1:2],
            batch['input_ids'][...,2::2]
        )
        out['loss'] = self.alpha * out.even_loss + out.odd_loss
        return out

    def _log_loss(self, out, prefix):
        for k in out.keys():
            if 'loss' in k:
                self.log(prefix + '_' + k, out[k].item())

    def training_step(self, batch, batch_idx):
        out = self.forward(batch)
        self._log_loss(out, 'train')
        return out.loss

    def validation_step(self, batch, batch_idx):
        out = self.forward(batch)
        self._log_loss(out, 'val')
        return out.loss

    def test_step(self, batch, batch_idx):
        out = self.forward(batch)
        self._log_loss(out, 'test')
        return out.loss
        
    def configure_optimizers(self):
        optimizer = optim.Adam(
            params=self.model.parameters(),
            lr=self.lr,
        )
        scheduler = transformers.get_cosine_schedule_with_warmup(
            optimizer=optimizer,
            num_warmup_steps=self.num_warmup_steps,
            num_training_steps=9200 #1 epoch #self.trainer.estimated_stepping_batches,
        )
        return (
            [optimizer],
            [{"scheduler": scheduler, "interval": "step"}]
        )

In [5]:
NAME = 'GPT2-UNBALANCED'
PROJ = 'LAISR_FUTURE_GPT2'

In [6]:
train = datasets.load_from_disk('/home/wwu/msmarco_GPT2_64tokens_full/train').with_format('torch')
val = datasets.load_from_disk('/home/wwu/msmarco_GPT2_64tokens_full/val').with_format('torch')
train_loader = DataLoader(train, batch_size=512, num_workers=251)
val_loader = DataLoader(val, batch_size=512, num_workers=111)

In [7]:
wandb_logger = WandbLogger(
    name=NAME,
    project=PROJ,
    log_model=False,   # Only save checkpoints locally
)

In [8]:
lr_monitor = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(
    dirpath="/home/wwu/checkpoints",
    filename=NAME + "_{global_step}_{val_loss:.2f}",
    every_n_epochs=1,
    save_top_k=1,
    monitor='val_loss',
    mode='min',
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    divergence_threshold=15,
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode='min',
)
trainer = L.Trainer(
    fast_dev_run=False,
    logger=wandb_logger,
    val_check_interval=.1,
    callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
    max_epochs=1,
    enable_progress_bar=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
model = LitUnbalancedModel()
wandb_logger.watch(model.model, log='all')

wandb: Currently logged in as: wilswu. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [10]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,#loaders['train'],
    val_dataloaders=val_loader,#loaders['val']
)

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /home/wwu/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 124 M 
------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
497.759   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

